In [3]:
import sympy as sy
import math
from sympy import *
import numpy as np


In [4]:
def DHmatrix(theta, d, a, alpha):
    '''
    根据机器人的DH参数求连杆变换矩阵
    '''

#     DH1 = [cos(theta), -sin(theta) * cos(alpha), sin(theta) * sin(alpha), a * cos(theta)]
#     DH2 = [sin(theta),cos(theta) * cos(alpha), -cos(theta) * sin(alpha), a * sin(theta)]
#     DH3 = [0, sin(alpha), cos(alpha), d]
#     DH4 = [0, 0, 0, 1]
# 书上的公式
    DH1 = [cos(theta), -sin(theta), 0, a]
    DH2 = [sin(theta) * cos(alpha), cos(theta) * cos(alpha) , -sin(alpha), -d * sin(alpha)]
    DH3 = [sin(theta) * sin(alpha), cos(theta) * sin(alpha), cos(alpha), d * cos(alpha)]
    DH4 = [0, 0, 0, 1]
    DH = np.array([DH1, DH2, DH3, DH4])
    return DH

In [8]:
def yacobi(theta, d, a, alpha):

    T01 = DHmatrix(theta[0], d[0], a[0], alpha[0])
    T12 = DHmatrix(theta[1], d[1], a[1], alpha[1])
    T23 = DHmatrix(theta[2], d[2], a[2], alpha[2])
    T34 = DHmatrix(theta[3], d[3], a[3], alpha[3])
    T45 = DHmatrix(theta[4], d[4], a[4], alpha[4])
    T56 = DHmatrix(theta[5], d[5], a[5], alpha[5])

    T01 = T01
    T02 = np.dot(T01, T12)
    T03 = np.dot(T02, T23)
    T04 = np.dot(T03, T34)
    T05 = np.dot(T04, T45)
    T06 = np.dot(T05, T56)

    R01 = T01[:3, :3]
    R12 = T12[:3, :3]
    R23 = T23[:3, :3]
    R34 = T34[:3, :3]
    R45 = T45[:3, :3]
    R56 = T56[:3, :3]

    R01 = R01
    R02 = np.dot(R01, R12)
    R03 = np.dot(R02, R23)
    R04 = np.dot(R03, R34)
    R05 = np.dot(R04, R45)
    R06 = np.dot(R05, R56)

    z1 = R01[2,:]
    z2 = R02[2,:]
    z3 = R03[2, :]
    z4 = R04[2, :]
    z5 = R05[2, :]
    z6 = R06[2, :]

    T56 = T56
    T46 = np.dot(T45, T56)
    T36 = np.dot(T34, T46)
    T26 = np.dot(T23, T36)
    T16 = np.dot(T12, T26)

    p66 = np.array([0, 0, 0])
    p56 = T56[:3, 3]
    p46 = T46[:3, 3]
    p36 = T36[:3, 3]
    p26 = T26[:3, 3]
    p16 = T16[:3, 3]

    J6 = np.cross(z6, np.dot(R06, p66))
    J5 = np.cross(z5, np.dot(R05, p56))
    J4 = np.cross(z4, np.dot(R04, p46))
    J3 = np.cross(z3, np.dot(R03, p36))
    J2 = np.cross(z2, np.dot(R02, p26))
    J1 = np.cross(z1, np.dot(R01, p16))

    J6 = np.concatenate((J6, z6))
    J5 = np.concatenate((J5, z5))
    J4 = np.concatenate((J4, z4))
    J3 = np.concatenate((J3, z3))
    J2 = np.concatenate((J2, z2))
    J1 = np.concatenate((J1, z1))

    yacobi = np.array([J1, J2, J3, J4, J5, J6])

    return yacobi

In [6]:
theta1,theta2,theta3,theta4,theta5,theta6 = sy.symbols('theta1,theta2,theta3,theta4,theta5,theta6')
a1,a2,a3,a4,a5,d1,d2,d3,d4,d5=sy.symbols('a1,a2,a3,a4,a5,d1,d2,d3,d4,d5')

In [6]:
# 论文上UR的参数
# T01 = DHmatrix(theta1, 0, 0, -pi/2)
# T12 = DHmatrix(theta2, d2, a2, 0)
# T23 = DHmatrix(theta3, d3, a3, 0)
# T34 = DHmatrix(theta4, 0, 0, pi/2)
# T45 = DHmatrix(theta5, d5, 0, pi/2)
# T56 = DHmatrix(theta6, 0, 0, 0)
# 书上puma560
T01 = DHmatrix(theta1, 0, 0, 0)
T12 = DHmatrix(theta2, d2, 0, -pi/2)
T23 = DHmatrix(theta3, 0, a2, 0)
T34 = DHmatrix(theta4, d4, a3, -pi/2)
T45 = DHmatrix(theta5, 0, 0, pi/2)
T56 = DHmatrix(theta6, 0, 0, -pi/2)
T01, T12, T23, T34, T45, T56

(array([[cos(theta1), -sin(theta1), 0, 0],
        [sin(theta1), cos(theta1), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta2), -sin(theta2), 0, 0],
        [0, 0, 1, d2],
        [-sin(theta2), -cos(theta2), 0, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta3), -sin(theta3), 0, a2],
        [sin(theta3), cos(theta3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta4), -sin(theta4), 0, a3],
        [0, 0, 1, d4],
        [-sin(theta4), -cos(theta4), 0, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta5), -sin(theta5), 0, 0],
        [0, 0, -1, 0],
        [sin(theta5), cos(theta5), 0, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta6), -sin(theta6), 0, 0],
        [0, 0, 1, 0],
        [-sin(theta6), -cos(theta6), 0, 0],
        [0, 0, 0, 1]], dtype=object))

In [35]:
T01 = T01
T02 = np.dot(T01, T12)
T03 = np.dot(T02, T23)
T04 = np.dot(T03, T34)
T05 = np.dot(T04, T45)
T06 = np.dot(T05, T56)

R01 = T01[:3, :3]
R12 = T12[:3, :3]
R23 = T23[:3, :3]
R34 = T34[:3, :3]
R45 = T45[:3, :3]
R56 = T56[:3, :3]

R01 = R01
R02 = np.dot(R01, R12)
R03 = np.dot(R02, R23)
R04 = np.dot(R03, R34)
R05 = np.dot(R04, R45)
R06 = np.dot(R05, R56)

z1 = R01[2,:]
z2 = R02[2,:]
z3 = R03[2, :]
z4 = R04[2, :]
z5 = R05[2, :]
z6 = R06[2, :]

T56 = T56
T46 = np.dot(T45, T56)
T36 = np.dot(T34, T46)
T26 = np.dot(T23, T36)
T16 = np.dot(T12, T26)

p66 = np.array([0, 0, 0])
p56 = T56[:3, 3]
p46 = T46[:3, 3]
p36 = T36[:3, 3]
p26 = T26[:3, 3]
p16 = T16[:3, 3]

J6 = np.cross(z6, np.dot(R06, p66))
J5 = np.cross(z5, np.dot(R05, p56))
J4 = np.cross(z4, np.dot(R04, p46))
J3 = np.cross(z3, np.dot(R03, p36))
J2 = np.cross(z2, np.dot(R02, p26))
J1 = np.cross(z1, np.dot(R01, p16))

J6 = np.concatenate((J6, z6))
J5 = np.concatenate((J5, z5))
J4 = np.concatenate((J4, z4))
J3 = np.concatenate((J3, z3))
J2 = np.concatenate((J2, z2))
J1 = np.concatenate((J1, z1))
simplify(z1), simplify(z2), simplify(z3), simplify(z4), simplify(z5), simplify(z6)
# simplify(p16), simplify(p26), simplify(p36), simplify(p46), simplify(p56), simplify(p66)
# np.dot(R04, p46), np.dot(R05, p56),np.dot(R06, p66)
# simplify(J1), simplify(J2), simplify(J3), simplify(J4), simplify(J5), simplify(J6)
# [J1,J2,J3,J4,J5,J6]
# simplify(J1)

([0, 0, 1],
 [-sin(theta2), -cos(theta2), 0],
 [-sin(theta2 + theta3), -cos(theta2 + theta3), 0],
 [-sin(theta2 + theta3)*cos(theta4), sin(theta4)*sin(theta2 + theta3), -cos(theta2 + theta3)],
 [-sin(theta5)*cos(theta2 + theta3) - sin(theta2 + theta3)*cos(theta4)*cos(theta5), sin(theta5)*sin(theta2 + theta3)*cos(theta4) - cos(theta5)*cos(theta2 + theta3), -sin(theta4)*sin(theta2 + theta3)],
 [-(sin(theta5)*cos(theta2 + theta3) + sin(theta2 + theta3)*cos(theta4)*cos(theta5))*cos(theta6) + sin(theta4)*sin(theta6)*sin(theta2 + theta3), (sin(theta5)*cos(theta2 + theta3) + sin(theta2 + theta3)*cos(theta4)*cos(theta5))*sin(theta6) + sin(theta4)*sin(theta2 + theta3)*cos(theta6), sin(theta5)*sin(theta2 + theta3)*cos(theta4) - cos(theta5)*cos(theta2 + theta3)])

In [24]:
simplify(np.dot(R01,R12))

[[cos(theta1)*cos(theta2), -sin(theta2)*cos(theta1), -sin(theta1)], [sin(theta1)*cos(theta2), -sin(theta1)*sin(theta2), cos(theta1)], [-sin(theta2), -cos(theta2), 0]]

In [11]:
a = [0, 0, a2, a3, 0, 0]
d = [0, d2, 0, d4, 0, 0]
alpha = [0, -pi/2, 0, -pi/2, pi/2, -pi/2]
theta = [theta1,theta2,theta3,theta4,theta5,theta6]

In [13]:
    T01 = DHmatrix(theta[0], d[0], a[0], alpha[0])
    T12 = DHmatrix(theta[1], d[1], a[1], alpha[1])
    T23 = DHmatrix(theta[2], d[2], a[2], alpha[2])
    T34 = DHmatrix(theta[3], d[3], a[3], alpha[3])
    T45 = DHmatrix(theta[4], d[4], a[4], alpha[4])
    T56 = DHmatrix(theta[5], d[5], a[5], alpha[5])
    T01,T12,T23,T34,T45,T56
    

(array([[cos(theta1), -sin(theta1), 0, 0],
        [sin(theta1), cos(theta1), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta2), -sin(theta2), 0, 0],
        [0, 0, 1, d2],
        [-sin(theta2), -cos(theta2), 0, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta3), -sin(theta3), 0, a2],
        [sin(theta3), cos(theta3), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta4), -sin(theta4), 0, a3],
        [0, 0, 1, d4],
        [-sin(theta4), -cos(theta4), 0, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta5), -sin(theta5), 0, 0],
        [0, 0, -1, 0],
        [sin(theta5), cos(theta5), 0, 0],
        [0, 0, 0, 1]], dtype=object),
 array([[cos(theta6), -sin(theta6), 0, 0],
        [0, 0, 1, 0],
        [-sin(theta6), -cos(theta6), 0, 0],
        [0, 0, 0, 1]], dtype=object))

In [37]:
R01 = T01[:3, :3]
R12 = T12[:3, :3]
R23 = T23[:3, :3]
R34 = T34[:3, :3]
R45 = T45[:3, :3]
R56 = T56[:3, :3]
R01,R12,R23,R34,R45,R56


[[cos(theta4), -sin(theta4), 0], [0, 0, 1], [-sin(theta4), -cos(theta4), 0]]

In [42]:
R01 = R01
R02 = np.dot(R01, R12)
R03 = np.dot(R02, R23)
R04 = np.dot(R03, R34)
R05 = np.dot(R04, R45)
R06 = np.dot(R05, R56)
R01,R02,R03,R04,R05,R06
simplify(R04)
simplify(R04)


[[sin(theta1)*sin(theta4) + cos(theta1)*cos(theta4)*cos(theta2 + theta3), sin(theta1)*cos(theta4) - sin(theta4)*cos(theta1)*cos(theta2 + theta3), -sin(theta2 + theta3)*cos(theta1)], [sin(theta1)*cos(theta4)*cos(theta2 + theta3) - sin(theta4)*cos(theta1), -sin(theta1)*sin(theta4)*cos(theta2 + theta3) - cos(theta1)*cos(theta4), -sin(theta1)*sin(theta2 + theta3)], [-sin(theta2 + theta3)*cos(theta4), sin(theta4)*sin(theta2 + theta3), -cos(theta2 + theta3)]]

In [16]:
z1 = R01[2,:]
z2 = R02[2,:]
z3 = R03[2, :]
z4 = R04[2, :]
z5 = R05[2, :]
z6 = R06[2, :]
z1,z2,z3,z4,z5,z6

(array([0, 0, 1], dtype=object),
 array([-sin(theta2), -cos(theta2), 0], dtype=object),
 array([-sin(theta2)*cos(theta3) - sin(theta3)*cos(theta2),
        sin(theta2)*sin(theta3) - cos(theta2)*cos(theta3), 0], dtype=object),
 array([(-sin(theta2)*cos(theta3) - sin(theta3)*cos(theta2))*cos(theta4),
        -(-sin(theta2)*cos(theta3) - sin(theta3)*cos(theta2))*sin(theta4),
        sin(theta2)*sin(theta3) - cos(theta2)*cos(theta3)], dtype=object),
 array([(sin(theta2)*sin(theta3) - cos(theta2)*cos(theta3))*sin(theta5) + (-sin(theta2)*cos(theta3) - sin(theta3)*cos(theta2))*cos(theta4)*cos(theta5),
        (sin(theta2)*sin(theta3) - cos(theta2)*cos(theta3))*cos(theta5) - (-sin(theta2)*cos(theta3) - sin(theta3)*cos(theta2))*sin(theta5)*cos(theta4),
        (-sin(theta2)*cos(theta3) - sin(theta3)*cos(theta2))*sin(theta4)],
       dtype=object),
 array([((sin(theta2)*sin(theta3) - cos(theta2)*cos(theta3))*sin(theta5) + (-sin(theta2)*cos(theta3) - sin(theta3)*cos(theta2))*cos(theta4)*cos(theta

In [19]:
T56 = T56
T46 = np.dot(T45, T56)
T36 = np.dot(T34, T46)
T26 = np.dot(T23, T36)
T16 = np.dot(T12, T26)

p66 = np.array([0, 0, 0])
p56 = T56[:3, 3]
p46 = T46[:3, 3]
p36 = T36[:3, 3]
p26 = T26[:3, 3]
p16 = T16[:3, 3]
p16,p26,p36,p46,p56,p66

(array([-(a3*sin(theta3) + d4*cos(theta3))*sin(theta2) + (a2 + a3*cos(theta3) - d4*sin(theta3))*cos(theta2),
        d2,
        -(a3*sin(theta3) + d4*cos(theta3))*cos(theta2) - (a2 + a3*cos(theta3) - d4*sin(theta3))*sin(theta2)],
       dtype=object),
 array([a2 + a3*cos(theta3) - d4*sin(theta3),
        a3*sin(theta3) + d4*cos(theta3), 0], dtype=object),
 array([a3, d4, 0], dtype=object),
 array([0, 0, 0], dtype=object),
 array([0, 0, 0], dtype=object),
 array([0, 0, 0]))

In [20]:
simplify(p16)

[a2*cos(theta2) + a3*cos(theta2 + theta3) - d4*sin(theta2 + theta3), d2, -a2*sin(theta2) - a3*sin(theta2 + theta3) - d4*cos(theta2 + theta3)]

In [44]:
    z1 = R01[:, 2]
    z2 = R02[:, 2]
    z3 = R03[:, 2]
    z4 = R04[:, 2]
    z5 = R05[:, 2]
    z6 = R06[:, 2]
    z1,z2,z3
    z4,z5,z6


(array([-sin(theta2)*cos(theta1)*cos(theta3) - sin(theta3)*cos(theta1)*cos(theta2),
        -sin(theta1)*sin(theta2)*cos(theta3) - sin(theta1)*sin(theta3)*cos(theta2),
        sin(theta2)*sin(theta3) - cos(theta2)*cos(theta3)], dtype=object),
 array([(-sin(theta2)*sin(theta3)*cos(theta1) + cos(theta1)*cos(theta2)*cos(theta3))*sin(theta4) - sin(theta1)*cos(theta4),
        (-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3))*sin(theta4) + cos(theta1)*cos(theta4),
        (-sin(theta2)*cos(theta3) - sin(theta3)*cos(theta2))*sin(theta4)],
       dtype=object),
 array([-((-sin(theta2)*sin(theta3)*cos(theta1) + cos(theta1)*cos(theta2)*cos(theta3))*cos(theta4) + sin(theta1)*sin(theta4))*sin(theta5) + (-sin(theta2)*cos(theta1)*cos(theta3) - sin(theta3)*cos(theta1)*cos(theta2))*cos(theta5),
        -((-sin(theta1)*sin(theta2)*sin(theta3) + sin(theta1)*cos(theta2)*cos(theta3))*cos(theta4) - sin(theta4)*cos(theta1))*sin(theta5) + (-sin(theta1)*sin(theta2)*cos(theta3) - si